# Извлечение данных с сайта
**Сбор информации о ресторанах с сайта при помощи технологии веб-скрапинга**  
*Инструменты: requests, pandas, BeutifulSoup, tqdm*

Большинство проектов начинается со сбора данных. Данные могут быть полученны из базы данных, Интернета, онлайн или офлайн опросах.  
В этой работе мы рассмотрим веб-скрапинг - способ извлечения данных с веб-страницы, на примере сайта restoclub.ru.

### Подключение библиотек

Для получения доступа к веб-сайту и извлечения данных с помощью Python будем ипользовать следующие библиотеки:  
- requests - позволит нам отправить HTTP запрос к сайту
- BeautifulSoup - позволит извлечь информацию из HTML файла
- pandas - для создания DataFrame и сохранения данных в файл CSV
- tqdm - добавит индикатор прогресса выполнения

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm

### Подключение к сайту

Сохраним адрес сайта в переменную

In [2]:
# адрес сайта
URL = 'https://www.restoclub.ru'
# адрес страницы с рейтингами ресторанов Санкт-Петербурга
best_rest_url = URL + '/spb/ratings'

Некоторые сайты определяют, что запрос пришел не от браузера и блокируют его. Что бы этого избежать подменим User-Agent в заголовке запроса

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}

Подключимся к сайту и проверим подключение

In [4]:
response = requests.get(best_rest_url, headers=headers)
response.status_code

200

Код 200 говорит об успешном подключении

Передадим соержимое страницы в BeautifulSoup, указав html.parser в качестве анализатора

In [5]:
content = response.content
soup = BeautifulSoup(content,'html.parser')

Теперь в переменной *soup* находится объект *BeautifulSoup* содержащий содержимое страницы с рейтингами ресторанов. Для составления рейтинга нам требуется пройтись по всем рейтингам.  
Для этого найдем на странице ссылки на подстраницы с рейтингами.

С помощью веб-инспектора определим, что нужные ссылки имеют класс *"btn _full _round _transparent _red-text more-btn"*. Найдем на странице все теги *a* с этим классом и сохраним сожержимое ссылки.

In [6]:
best_rest_urls = [x['href'] for x in soup.find_all('a', class_='btn _full _round _transparent _red-text more-btn')]
# добавим к ссылке адрес сайта
best_rest_urls = [URL + x for x in best_rest_urls]

Посмотрим, что получилось

In [7]:
best_rest_urls

['https://www.restoclub.ru/spb/ratings/russkaja-kuhnja-top-10',
 'https://www.restoclub.ru/spb/ratings/authors-cuisine',
 'https://www.restoclub.ru/spb/ratings/rejting-italjanskih-restoranov',
 'https://www.restoclub.ru/spb/ratings/rejting-vostochnyh-zavedenij',
 'https://www.restoclub.ru/spb/ratings/rejting-kavkazskih-zavedenij',
 'https://www.restoclub.ru/spb/ratings/rejting-japonskih-mest',
 'https://www.restoclub.ru/spb/ratings/rejting-kitajskih-zavedenij',
 'https://www.restoclub.ru/spb/ratings/luchshie-iz-nedorogih',
 'https://www.restoclub.ru/spb/ratings/reiting-vegetarianskih-kafe-i-restoranov',
 'https://www.restoclub.ru/spb/ratings/rejting-zagorodnyh-restoranov',
 'https://www.restoclub.ru/spb/ratings/reiting-kofeen-v-peterburge',
 'https://www.restoclub.ru/spb/ratings/vinoteki-i-vinnye-restorany',
 'https://www.restoclub.ru/spb/ratings/rejting-barov-peterburga',
 'https://www.restoclub.ru/spb/ratings/rejting-pivnyh-zavedenij',
 'https://www.restoclub.ru/spb/ratings/rejting-v

### Поиск подробных сведений

Теперь, когда список рейтингов готов, необходимо найти подробные сведения о каждом конкретном ресторане.  
В этой работе мы будем искать и сохранять следующие сведения:
- Название рейтинга
- Название ресторана
- Тип кухни
- Средний чек
- Рейтинг

Для этого возьмем первый рейтинг и попробуем найти нужные нам сведения у первого ресторана.

Для начала подключимся к 1 странице с рейтингом и создадим объект *BeautifylSoup*

In [8]:
best_rest_content = requests.get(best_rest_urls[0], headers=headers).content
best_rest_soup = BeautifulSoup(best_rest_content, 'html.parser')

С помощью веб-инспектора найдем тег, содержащий название рейтинга

In [9]:
best_rest_soup.find('div', class_='page__item page-ratings__description').h1.text

'Русские рестораны'

Аналогично найдем все остальные сведения

In [10]:
# найдем список ресторанов
rest_list = best_rest_soup.find('ul',attrs={'class': 'rating-places _full'})
# найдем все рестораны в списке
rest_items = rest_list.find_all('li', attrs={'class': 'rating-places__item'})

In [11]:
# найдем название ресторана
rest_items[0].find('span', attrs={'class': 'search-place-title__name'}).find('span').text

'Проспект'

In [12]:
# тип кухни
[x.text for x in rest_items[0].find('li', class_='search-place-card__info-item _cuisine').find_all('span', class_='cuisine')]

['русская', 'европейская']

In [13]:
# средний чек
print(rest_items[0].find('li', class_='search-place-card__info-item _bill').text)

1050 


In [14]:
# рейтинг
rest_items[0].find('div', attrs={'class': 'rating__value'}).text

'9.4'

### Сохранение в таблицу 

Мы нашли все теги, содержащие нужные нам данные, теперь можно применить цикл и сохранить нужную инфорацию со всех страниц.  
Для наблюдения за процессом добавим индикатор прогресса с помощью *tqdm*

In [15]:
list_rest = []
for list in tqdm(best_rest_urls):
    best_rest_content = requests.get(list, headers=headers).content
    best_rest_soup = BeautifulSoup(best_rest_content, 'html.parser')
    rest_list = best_rest_soup.find('ul',attrs={'class': 'rating-places _full'})
    rest_items = rest_list.find_all('li', attrs={'class': 'rating-places__item'})
    for item in tqdm(rest_items, leave=None):
        df = {}
        df['list_name'] = best_rest_soup.find('div', class_='page__item page-ratings__description').h1.text
        df['rest_name'] = item.find('span', attrs={'class': 'search-place-title__name'}).find('span').text
        # так как у некоторых ресторанов отсутствуют некоторые сведения,
        # используем конструкцию try-except
        try:
            df['price'] = item.find('li', class_='search-place-card__info-item _bill').text
        except:
            df['price'] = 0
        try:
            df['cuisine'] = [x.text for x in item.find('li', class_='search-place-card__info-item _cuisine').find_all('span', class_='cuisine')]
        except:
            df['cuisine'] = 'нет данных'
        df['rating'] = item.find('div', attrs={'class': 'rating__value'}).text
        list_rest.append(df)

  0%|          | 0/21 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Полученый список преобразуем в DataFrame

In [16]:
list_rest = pd.DataFrame(list_rest)

Посмотрим результат

In [17]:
list_rest

,list_name,rest_name,price,cuisine,rating
0,Русские рестораны,Проспект,1050,"[русская, европейская]",9.4
1,Русские рестораны,Тройка,1750,"[европейская, русская]",9.2
2,Русские рестораны,Usoff,1300,"[европейская, русская]",9.2
3,Русские рестораны,Мари Vanna,2500,"[русская, вино]",9.1
4,Русские рестораны,Сытинъ,1200,"[русская, корюшка]",9
...,...,...,...,...,...
205,Семейные рестораны,Сказкино варенье,700,"[русская, европейская, пицца]",8.9
206,Семейные рестораны,Артишок,900,"[европейская, смешанная]",8.9
207,Семейные рестораны,Вкусновица,1500,"[сербская, вино]",8.8
208,Семейные рестораны,Лесопилка,1100,"[европейская, итальянская]",8.8


Сохраним полученную таблицу в файл

In [18]:
list_rest.to_csv('restorans.csv', index=False)

### Вывод
С помощью веб-скрапинга сайта с рейтингами ресторанов мы получили таблицу, содержащую информацию о лучших ресторанах Санкт-Петербурга.  
Полученная таблица была сохранена в файл формата CSV для последующего анализа